In [1]:
#Imports 

import sys
import os
import yaml

sys.path.append(os.getenv("CODE_PATH"))
sys.path.append(os.getenv("FIN_DATABASE_PATH"))


import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import logging
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from Data.connect import engine, DailyStockData, HourlyStockData, OneMinuteStockData, FiveMinuteStockData,FifteenMinuteStockData, StockSplits, StockNews, CompanyFinancials
from Pre_Processing.pre_processing import PreProcessing
from data_fetcher import DataFetcher
from Feature_Engineering.feature_engineering import TechnicalIndicators
from pipeline import Pipeline
import json
from pandas import json_normalize

In [2]:
GENERAL_COLUMNS = ['company_name', 'start_date', 'end_date', 'filing_date',
       'fiscal_period', 'fiscal_year', 'acceptance_datetime',
       'timeframe', 'tickers', 'sic']
SECTIONS = ['balance_sheet', 'income_statement', 'cash_flow_statement', 'comprehensive_income']


In [3]:
#Selecting some tickers to analyse
tickers = ['AAPL', 'MSFT']

In [4]:
class PreProcessingFinancials:
    def __init__(self, data, sections, tickers):
        """
        args: 
        data: dictionary with multiple tickers
        sections: list with sections from a Financial Statement
        tickers: list with tickers we want to analyse
        """
        if isinstance(data,pd.DataFrame): #If we pass only one ticker, which would be only one dataframe, we transform into a dictionary.
            self.data = {}
            self.data[tickers] = data
            self.tickers = [tickers]
        else:
            self.data = data
            self.tickers = tickers 
        self.sections = sections
        
    def adjust_data(self, data):
        """Adjusting filling date for TTM, Q4, FY, as they don't have any filing dates.
        Using as a proxy the end date of the period plus 37 days, which is the average time it takes to file the 10-K/10-Q."""
        conditions = (
            (data['filing_date'].isna()) &
            (data['fiscal_period'].isin(['TTM', 'Q4', 'FY']))
                        )
    
        data['filing_date'] = np.where(conditions, data['end_date'] + timedelta(days=37), data['filing_date'])
      
        return data
    
    def replacing_nan(self,data):
        """Replace NaN values in the fiscal year column with the correct date."""
        data['period'] = np.where(data['fiscal_'])
        
    
    def flatten_json_section(self):
        """Preprocess the financials column by flattening JSON fields and handling filing dates/fiscal periods."""
        for ticker in self.tickers:
            data = self.data[ticker]
            
            # Converting JSON strings into Python dictionaries if necessary
            data['financials'] = data['financials'].apply(
                lambda x: json.loads(x) if isinstance(x, str) else x
            )

            # Flattening each section and concatenate with the main dataframe
            for section in self.sections:
                flattened = self.flattening(data, 'financials', section)
                data = pd.concat([data, flattened], axis=1)

            # Handle filing dates and fiscal periods
            data['filing_date'] = pd.to_datetime(data['filing_date'])

            #Adjust filing dates before setting index
            data = self.adjust_data(data)

            #Sort by filing date 
            data.sort_values(by='filing_date', inplace=True, ascending=False)

            self.data[ticker] = data
    
    
    
    def flattening(self, data, json_col, section):
        """ Helper function to flatten a JSON section of the financials dataframe
        """
        section_data = data[json_col].apply(lambda x: x.get(section) if section in x else {})
        flattened_section = json_normalize(section_data)  # Flatten the section
        flattened_section.columns = [f"{section}_{col}" for col in flattened_section.columns]  # Add prefix to columns
                
        return flattened_section

    
    def removing_cols(self):
        """ This function cleans the dataframe by dropping columns with '.unit' in the name and '.order' in the name.
        If .unit columns are the same for each row, we will drop.
        Also dropping columns that have .order
        """
        
        for ticker in self.tickers:
            data = self.data[ticker]
            # print(f'Processing ticker {ticker}')
            for section in self.sections:
                section_columns = [col for col in data.columns if section in col]
                section_df = data[section_columns].copy()
            
                #Removing .order columns
                order_columns = [col for col in section_df.columns if '.order' in col]
                # if order_columns:
                #     print(f"Found '.order' columns for {ticker} in {section}: {order_columns}")  # Debugging
                # else:
                #     print(f"No '.order' columns found for {ticker} in {section}") 
                section_df.drop(columns=order_columns, inplace=True)
                
                #Removing .unit columns if they only have one unique value
                unit_columns = [col for col in section_df.columns if '.unit' in col]
                # print(f"Found '.unit' columns for {ticker} in {section}: {unit_columns}")  # Debugging
 
                for col in unit_columns:
                    if section_df[col].nunique() == 1:
                        section_df.drop(columns=col, inplace=True)
        
                
                label_columns = [col for col in section_df.columns if '.label' in col]
                section_df.drop(columns=label_columns, inplace=True)
                
                
                #Converting numeric values to millions
                value_columns = [col for col in section_df.columns if '.value' in col]
                for col in value_columns:
                    section_df[col] = pd.to_numeric(section_df[col])
                    section_df[col] = section_df[col]/1000000
                    # print(f"Converted {col} to millions for {ticker} in {section}")  # Debugging
                data.drop(columns=section_columns, inplace=True)  # Remove the original section columns
                data = pd.concat([data, section_df], axis=1)
               
            self.data[ticker] = data
    
    def transform_columns(self, data):
        """Transform data column data into required type"""
        
        # Replacing empty strings with np.nan in the entire dataframe at once
        data.replace('', np.nan, inplace=True)
        
        # Convert specific columns to required types
        data['fiscal_year'] = pd.to_numeric(data['fiscal_year'], errors='coerce')
        data['fiscal_period'] = data['fiscal_period'].astype('category')
        data['start_date'] = pd.to_datetime(data['start_date'], errors='coerce')
        data['end_date'] = pd.to_datetime(data['end_date'], errors='coerce')
        
    def create_period(self, row):
        if row['timeframe'] == 'quarterly':
            quarter = int(row['fiscal_period'].replace('Q', ''))
            year = int(row['fiscal_year'])
            return pd.Period(freq='Q', year=year, quarter=quarter)
        else:
            return np.nan
        
        
    def preprocess_financials(self):
        """Orchestrates the entire pre-processing of financials."""
        self.flatten_json_section()  
        self.removing_cols() 
        
        # Step 3: Create multi-indexed DataFrame for each ticker
        processed_data = {}
        
        for ticker in self.tickers:
            data = self.data[ticker]
            
            # Extract general columns from the data
            general_df = data[GENERAL_COLUMNS].copy()

            # If 'filing_date' is not already an index, set it as the index in general_df
            if 'filing_date' in general_df.columns:
                general_df.set_index('filing_date', inplace=True)
            
            # Proccessing each section
            section_dataframes = []
            
           
            for section in self.sections:
                # Filter columns related to the current section
                section_columns = [col for col in data.columns if section in col]
                section_df = data[['filing_date'] + section_columns].copy()  # Ensure 'filing_date' is included

                # Set 'filing_date' as the index for the section to align it properly
                section_df.set_index('filing_date', inplace=True)

                # Removing '.value' suffix from the column names
                section_df.columns = section_df.columns.str.replace('.value', '', regex=False)

                # Removing the section name from the second-level column names
                section_df.columns = pd.MultiIndex.from_product(
                    [[section], section_df.columns.str.replace(f'{section}_', '', regex=False)]
                )

                # Add this section DataFrame to the list
                section_dataframes.append(section_df)
            
            # Concatenate all section DataFrames into one DataFrame (financial data)
            financial_data = pd.concat(section_dataframes, axis=1)

            # Concatenate general_df (with general columns) and financial_data (with sections)
            full_data = pd.concat([general_df, financial_data], axis=1)

            # Store the processed DataFrame
            processed_data[ticker] = full_data
            
        combined_data = pd.concat(processed_data.values(), keys=processed_data.keys(), names=['ticker'])
        self.transform_columns(combined_data)
        combined_data['period'] = combined_data.apply(self.create_period, axis=1)
        
        
        return processed_data, combined_data

In [5]:
class CalculateMetrics:
    
    def __init__(self, data):
        """ data is a dataframe with tickers as index."""
        self.data = data
        
    def profitability_ratios(self):
        """Calculates profitability ratios."""
        # data = self.self.data.copy()
        
        # Gross Margin
        self.data['gross_margin'] = self.data[('income_statement', 'gross_profit')] / self.data[('income_statement', 'revenues')]
        
        # Operating Margin
        self.data['operating_margin'] = self.data[('income_statement', 'operating_income_loss')] / self.data[('income_statement', 'revenues')]
        
        # Net Profit Margin
        self.data['net_profit_margin'] = self.data[('income_statement', 'net_income_loss')] / self.data[('income_statement', 'revenues')]
        
        #ROA
        self.data['ROA'] = self.data[('income_statement', 'net_income_loss')] / self.data[('balance_sheet', 'assets')]
        
        #ROE
        self.data['ROE'] = self.data[('income_statement', 'net_income_loss')] / self.data[('balance_sheet', 'equity')]
        
        return self.data
    
    def liquidity_ratios(self):
        """ Calculates liquidity ratios."""
        # data = self.self.data.copy()

        #Current Ratio
        self.data['current_ratio'] = self.data[('balance_sheet', 'current_assets')] / self.data[('balance_sheet', 'current_liabilities')]
        
        #Quick Ratio
        self.data['quick_ratio'] = (self.data[('balance_sheet', 'current_assets')] - self.data[('balance_sheet', 'inventory')]) / self.data[('balance_sheet', 'current_liabilities')]
        
    
    def other_ratios(self):
        """Calculates other ratios."""
        # data = self.self.data.copy()
        
        #Debt to Equity
        self.data['debt_to_equity'] = self.data[('balance_sheet', 'liabilities')] / self.data[('balance_sheet', 'equity')]
        
        #Interest Coverage
        self.data['interest_coverage'] = self.data[('income_statement', 'operating_income_loss')] / self.data[('income_statement', 'interest_expense_operating')]
        self.data['R&D_ratio'] = self.data[('income_statement', 'research_and_development')] / self.data[('income_statement', 'revenues')]
        
        return self.data
    
    def calculate_metrics(self):
        """Orchestrates the calculation of financial metrics."""
        self.profitability_ratios()
        self.liquidity_ratios()
        self.other_ratios()
        
        return self.data
        

In [6]:
#Fetching data from our SQL database

fetch_data = DataFetcher(tickers)
company_data = fetch_data.get_company_data()

In [7]:
#Pre Processing the data

prepocess = PreProcessingFinancials(company_data, SECTIONS, tickers)
data_dict, df = prepocess.preprocess_financials()

In [8]:
#Calculating Financial ratios

metrics = CalculateMetrics(df)
final_data = metrics.calculate_metrics()

In [23]:
final_data[final_data['timeframe'] == 'quarterly'].sort_values('period')

company_name start_date   end_date fiscal_period  \
ticker filing_date                                                       
AAPL   2009-07-22        APPLE INC 2009-03-29 2009-06-27            Q3   
       2009-11-02        APPLE INC 2009-06-28 2009-09-26            Q4   
       2010-01-25        APPLE INC 2009-09-27 2009-12-26            Q1   
MSFT   2009-10-23   MICROSOFT CORP 2009-07-01 2009-09-30            Q1   
       2010-01-28   MICROSOFT CORP 2009-10-01 2009-12-31            Q2   
...                            ...        ...        ...           ...   
       2024-01-30   MICROSOFT CORP 2023-10-01 2023-12-31            Q2   
AAPL   2024-05-03       Apple Inc. 2023-12-31 2024-03-30            Q2   
MSFT   2024-04-25   MICROSOFT CORP 2024-01-01 2024-03-31            Q3   
AAPL   2024-08-02       Apple Inc. 2024-03-31 2024-06-29            Q3   
MSFT   2024-08-06   MICROSOFT CORP 2024-04-01 2024-06-30            Q4   

                    fiscal_year acceptance_datetime  timeframe tickers   sic  \
ticker filing_date                                                             
AAPL   2009-07-22        2009.0                None  quarterly    AAPL  3571   
       2009-11-02        2009.0                None  quarterly    AAPL  3571   
       2010-01-25        2010.0                None  quarterly    AAPL  3571   
MSFT   2009-10-23        2010.0                None  quarterly    MSFT  7372   
       2010-01-28        2010.0                None  quarterly    MSFT  7372   
...                         ...                 ...        ...     ...   ...   
       2024-01-30        2024.0                None  quarterly    MSFT  7372   
AAPL   2024-05-03        2024.0                None  quarterly    AAPL  3571   
MSFT   2024-04-25        2024.0                None  quarterly    MSFT  7372   
AAPL   2024-08-02        2024.0                None  quarterly    AAPL  3571   
MSFT   2024-08-06        2024.0                None  quarterly    MSFT  7372   

                    (balance_sheet, noncurrent_liabilities)  ...  \
ticker filing_date                                           ...   
AAPL   2009-07-22                                    5591.0  ...   
       2009-11-02                                    6737.0  ...   
       2010-01-25                                    5061.0  ...   
MSFT   2009-10-23                                   11639.0  ...   
       2010-01-28                                   12098.0  ...   
...                                                     ...  ...   
       2024-01-30                                  111274.0  ...   
AAPL   2024-05-03                                  139395.0  ...   
MSFT   2024-04-25                                  112598.0  ...   
AAPL   2024-08-02                                  133280.0  ...   
MSFT   2024-08-06                                  118400.0  ...   

                    gross_margin  operating_margin  net_profit_margin  \
ticker filing_date                                                      
AAPL   2009-07-22       0.362600          0.200552           0.147415   
       2009-11-02       0.366160          0.222188           0.168693   
       2010-01-25       0.408787          0.301282           0.215392   
MSFT   2009-10-23       0.780031          0.346904           0.276625   
       2010-01-28       0.809273          0.447534           0.350226   
...                          ...               ...                ...   
       2024-01-30       0.683602          0.435859           0.352628   
AAPL   2024-05-03       0.465781          0.307428           0.260443   
MSFT   2024-04-25       0.700847          0.445876           0.354667   
AAPL   2024-08-02       0.462572          0.295557           0.250044   
MSFT   2024-08-06       0.695892          0.431427           0.340445   

                         ROA       ROE  current_ratio  quick_ratio  \
ticker filing_date                                                   
AAPL   2009-07-22   0.025530  0.047474     

In [11]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd

In [13]:
#Extracting unique tickers

tickers = final_data.index.get_level_values(0).unique()
timeframes = final_data['timeframe'].unique()
metrics = ['gross_margin',
           'operating_margin',
           'net_profit_margin',
           'ROA',
           'ROE',
           'current_ratio',
           'quick_ratio',
           'debt_to_equity',
           'interest_coverage',
           'R&D_ratio']
graph_types = ['Line', 'Bar', 'Scatter']



In [15]:
#Initialising tha dash app

app = dash.Dash(__name__)

#Defining our layout

app.layout = html.Div([
    html.H1("Financial Dashboard"), # our Header
    
    #Dropdown menu to select the ticker
    dcc.Dropdown(id='Select Ticker',
                options=[{'label': ticker, 'value': ticker} for ticker in tickers
                ],
                value=tickers[0], #default value set to the first ticker
                style = {'width': '50%'}
                ),
    
    
    
    #Dropdown to select a timeframe
    dcc.Dropdown(id='Select Timeframe',
                 options =[{'label': timeframe, 'value': timeframe} for timeframe in timeframes],
                 value = 'quarterly',
                 style = {'width': '50%'}
    ),
    
    #Dropdown to select metrics
    dcc.Dropdown(id='Select Metric',
             options=[{'label': metric, 'value': metric} for metric in metrics],
             value=metrics[0],  # default to the first metric
             style={'width': '50%'}
    ),
        

    # Add a new Dropdown for selecting graph type
    dcc.Dropdown(id='Select Graph Type',
                options=[{'label': graph_type, 'value': graph_type} for graph_type in graph_types],
                value='Line',  # default to Line chart
                style={'width': '50%'}
    ),

    
    dcc.Graph(id='financial-plot')
])

# if __name__ == '__main__':
#     app.run_server(debug=True)

In [16]:
@app.callback(
    Output('financial-plot', 'figure'),
    [Input('Select Ticker', 'value'),
     Input('Select Timeframe', 'value'),
     Input('Select Metric', 'value'),
     Input('Select Graph Type', 'value')]
)
def update_graph(selected_ticker, selected_timeframe, selected_metric, selected_graph_type):
    # Filter the dataframe based on the selected ticker
    filtered_df = final_data[(final_data.index.get_level_values(0) == selected_ticker)]
    
    # Filter the dataframe based on the selected timeframe
    if selected_timeframe == 'quarterly':
        # Filter only quarterly data
        filtered_df = filtered_df[filtered_df['timeframe'] == 'quarterly']
        # Use the 'period' column as the x-axis
        x_axis = filtered_df['period'].astype(str) 
    else:
        # For FY or TTM, use fiscal_year as the x-axis
        filtered_df = filtered_df[filtered_df['timeframe'] == selected_timeframe]
        x_axis = filtered_df['fiscal_year']

    # Create the figure based on the selected graph type
    if selected_graph_type == 'Line':
        fig = go.Figure(data=[
            go.Scatter(x=x_axis, y=filtered_df[selected_metric], mode='lines+markers')
        ])
    elif selected_graph_type == 'Bar':
        fig = go.Figure(data=[
            go.Bar(x=x_axis, y=filtered_df[selected_metric])
        ])
    elif selected_graph_type == 'Scatter':
        fig = go.Figure(data=[
            go.Scatter(x=x_axis, y=filtered_df[selected_metric], mode='markers')
        ])

    # Update the title and layout
    fig.update_layout(title=f"{selected_metric} for {selected_ticker} ({selected_timeframe})")
    return fig


In [17]:
if __name__ == '__main__':
    app.run_server(debug=True)
